In [ ]:
!pip install -q openai langchain langchainhub pypdf langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install chromadb

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00


In [ ]:
import os

from langchain.document_loaders import PyPDFLoader
import io
from PIL import Image
import IPython.display as display

from langchain.text_splitter import CharacterTextSplitter # Chunck 쪼개기

from langchain.embeddings.openai import OpenAIEmbeddings # 문서의 내용 임베딩
from langchain.vectorstores import Chroma # 벡터스토어에 저장

from langchain import hub # 프롬포트 템플릿

from langchain.chat_models import ChatOpenAI # 생성
from langchain.schema.runnable import RunnablePassthrough # Rag 체인 생성

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
loader = PyPDFLoader("/content/dat 프로젝트.pdf")
document = loader.load()
document[0].page_content[:500] # 내용 추출

' \n \n골다공증 (Osteoporosis) \n1. 정의 \n골다공증은 뼈의 밀도와 강도가 감소하여 뼈가 약해지고 쉽게 부러지는 질환입니다. 정\n상적인 뼈 구조가 점차적으로 소실되면서 골절의 위험이 크게 증가합니다. 주로 척추, 고\n관절(엉덩이뼈), 손목 등에서 골절이 잘 발생하며, 노인들에게 흔히 발생하는 만성 질환\n입니다. \n2. 원인 \n골다공증은 뼈를 형성하는 과정과 뼈를 흡수하는 과정의 불균형으로 발생합니다. 주요 \n원인은 다음과 같습니다: \n(노화: 나이가 들수록 뼈의 형성과 흡수 사이의 균형이 깨져 뼈가 약해짐, 호르몬 변화: \n폐경 후 여성에서 에스트로겐 감소로 뼈 손실 증가, 영양 결핍: 칼슘과 비타민 D 섭취 \n부족, 운동 부족: 체중 부하 운동의 부족으로 뼈 강도 저하, 유전적 요인: 가족 중 골다공\n증 병력이 있는 경우, 흡연 및 음주: 뼈 형성을 방해하고 골 손실을 촉진, 질병 및 약물: \n갑상선 기능 항진증, 류마티스 관절염, 만성 신장질환 및 스테로이드 장기 복'

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(document)

In [ ]:
texts

[Document(metadata={'source': '/content/dat 프로젝트.pdf', 'page': 0}, page_content='골다공증 (Osteoporosis) \n1. 정의 \n골다공증은 뼈의 밀도와 강도가 감소하여 뼈가 약해지고 쉽게 부러지는 질환입니다. 정\n상적인 뼈 구조가 점차적으로 소실되면서 골절의 위험이 크게 증가합니다. 주로 척추, 고\n관절(엉덩이뼈), 손목 등에서 골절이 잘 발생하며, 노인들에게 흔히 발생하는 만성 질환\n입니다. \n2. 원인 \n골다공증은 뼈를 형성하는 과정과 뼈를 흡수하는 과정의 불균형으로 발생합니다. 주요 \n원인은 다음과 같습니다: \n(노화: 나이가 들수록 뼈의 형성과 흡수 사이의 균형이 깨져 뼈가 약해짐, 호르몬 변화: \n폐경 후 여성에서 에스트로겐 감소로 뼈 손실 증가, 영양 결핍: 칼슘과 비타민 D 섭취 \n부족, 운동 부족: 체중 부하 운동의 부족으로 뼈 강도 저하, 유전적 요인: 가족 중 골다공\n증 병력이 있는 경우, 흡연 및 음주: 뼈 형성을 방해하고 골 손실을 촉진, 질병 및 약물: \n갑상선 기능 항진증, 류마티스 관절염, 만성 신장질환 및 스테로이드 장기 복용) \n3. 증상 \n골다공증은 조기에는 증상이 거의 없으며, “침묵의 질환”으로 불립니다. 그러나 뼈 손실\n이 진행되면 다음과 같은 증상이 나타날 수 있습니다: \n(골절(특히 척추, 손목, 고관절 골절), 키가 감소 (척추 압박 골절로 인한 경우), 허리 및 \n등 통증, 구부정한 자세(척추 후만증), 뼈가 약해져 작은 충격에도 골절 발생) \n4. 진단 방법 \n골다공증의 진단은 여러 방법을 통해 이루어집니다. 가장 기본적인 검사는 **골밀도 검\n사(BMD)**로, 이중 에너지 X선 흡수계측법(DXA)을 사용하여 뼈의 밀도를 측정합니다. \nDXA는 주로 척추(요추)와 고관절 부위를 검사하며, 손목 등의 골밀도도 평가할 수 있습\n니다. 검사 결과는 T-점수로 나타나며, -1.0 이상은 정상, -1.0에서 -2.5

In [ ]:
# 저장 및 검색

# 임베딩
embeddings = OpenAIEmbeddings()
# Chroma DB 에 저장
docsearch = Chroma.from_documents(texts, embeddings)
# retriever 가져옴
retriever = docsearch.as_retriever()

In [ ]:
# 템플릿 가져오기

rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# 생성

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

<ipython-input-24-9e959241c9fa>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

In [ ]:
rag_chain.invoke("대사 기능이 저하한 거 같아 무슨 질병이 의심 돼")

AIMessage(content='대사 기능 저하가 의심된다면 당뇨병이나 비만과 같은 대사 관련 질환이 있을 수 있습니다. 또한, 수면장애가 대사 기능에 영향을 미쳐 혈당 조절 문제를 유발할 수 있습니다. 이러한 증상은 조기 치료와 관리가 필요합니다.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 3160, 'total_tokens': 3229, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-5d5acbdf-5b71-4a5a-b482-88ca35f92fb7-0')

In [ ]:
response = rag_chain.invoke("수면이 부족한 문제가 있는데 어떤 질병이 의심 돼?")
content = response.content  # content만 추출

# 온점을 기준으로 문장을 나누고, 온점 뒤에 줄바꿈 추가
formatted_content = "\n".join([sentence.strip() + '.' for sentence in content.split('.') if sentence.strip()])

print(formatted_content)

수면 부족 문제는 여러 질병과 관련이 있을 수 있으며, 특히 수면장애가 의심됩니다.
대표적인 수면장애로는 불면증, 수면무호흡증, 하지불안증후군 등이 있으며, 이들은 신체적 및 정신적 건강에 부정적인 영향을 미칠 수 있습니다.
또한, 우울증이나 불안장애와 같은 정신적 문제도 수면 부족과 관련이 있을 수 있습니다.


In [ ]:
while True:
    question = input("어디가 아프신가요? (종료하려면 '종료' 입력): ")

    # 종료 조건
    if question == "종료":
        print("프로그램을 종료합니다.")
        break

    # 질문에 대해 답변을 얻음
    response = rag_chain.invoke(question)
    content = response.content  # content만 추출

    # 온점을 기준으로 문장을 나누고, 온점 뒤에 줄바꿈 추가
    formatted_content = "\n".join([sentence.strip() + '.' for sentence in content.split('.') if sentence.strip()])

    # 출력
    print("\n답변:\n")
    print(formatted_content)
    print("\n※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※\n")
    print("\n==============================\n")


어디가 아프신가요? (종료하려면 '종료' 입력): 무릎이 시큰거려

답변:

무릎이 시큰거리는 증상은 여러 원인에 의해 발생할 수 있습니다.
일반적으로 관절염, 부상, 또는 과사용 등이 원인이 될 수 있으며, 적절한 진단과 치료가 필요합니다.
증상이 지속되거나 심해지면 전문가의 상담을 받는 것이 좋습니다.

※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※



어디가 아프신가요? (종료하려면 '종료' 입력): 관절염은 어떤 증상이 있어?

답변:

관절염의 주요 증상으로는 관절의 통증, 부기, 경직, 그리고 운동 범위의 제한이 있습니다.
이러한 증상은 주로 아침에 심해지며, 활동 후에는 완화될 수 있습니다.
또한, 관절염은 피로감과 전신적인 불편함을 동반할 수 있습니다.

※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※



어디가 아프신가요? (종료하려면 '종료' 입력): 종료
프로그램을 종료합니다.


In [ ]:
previous_question = None
previous_answer = None

while True:
    question = input("어디가 아프신가요? (종료하려면 '종료' 입력): ")

    # 종료 조건
    if question == "종료":
        print("프로그램을 종료합니다.")
        break

    # 이전 질문과 답변을 context에 포함
    context = ""
    if previous_question and previous_answer:
        context = f"이전 질문: {previous_question}\n이전 답변: {previous_answer}\n\n"

    # 질문에 대해 답변을 얻음
    response = rag_chain.invoke(context + question)
    content = response.content  # content만 추출

    # 온점을 기준으로 문장을 나누고, 온점 뒤에 줄바꿈 추가
    formatted_content = "\n".join([sentence.strip() + '.' for sentence in content.split('.') if sentence.strip()])

    # 출력
    print("\n답변:\n")
    print(formatted_content)
    print("\n※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※\n")
    print("\n==============================\n")

    # 이전 질문과 답변을 저장
    previous_question = question
    previous_answer = formatted_content


어디가 아프신가요? (종료하려면 '종료' 입력): 골다공증인 거 같아

답변:

골다공증은 뼈의 밀도와 강도가 감소하여 뼈가 약해지고 쉽게 부러지는 질환입니다.
증상이 초기에는 거의 없지만, 진행되면 골절, 허리 통증, 키 감소 등의 증상이 나타날 수 있습니다.
골다공증이 의심된다면 골밀도 검사를 통해 진단받는 것이 중요합니다.

※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※



어디가 아프신가요? (종료하려면 '종료' 입력): 뭐가 원인이야?

답변:

골다공증의 주요 원인은 뼈를 형성하는 과정과 흡수하는 과정의 불균형입니다.
노화, 호르몬 변화, 영양 결핍, 운동 부족, 유전적 요인, 흡연 및 음주 등이 원인으로 작용할 수 있습니다.
이러한 요인들이 복합적으로 작용하여 뼈의 밀도와 강도가 감소하게 됩니다.

※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※



어디가 아프신가요? (종료하려면 '종료' 입력): 흡연을 좀 하긴 해

답변:

흡연은 골다공증의 원인 중 하나로, 뼈 형성을 방해하고 골 손실을 촉진합니다.
따라서 흡연을 줄이거나 중단하는 것이 뼈 건강에 도움이 될 수 있습니다.
골다공증 예방을 위해서는 금연과 함께 균형 잡힌 식사와 규칙적인 운동이 중요합니다.

※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※



어디가 아프신가요? (종료하려면 '종료' 입력): 골다공증인지 어떻게 알 수 있을까?

답변:

골다공증인지 확인하기 위해서는 여러 가지 검사를 통해 진단할 수 있습니다.
가장 기본적인 방법은 골밀도 검사(BMD)로, 이중 에너지 X선 흡수계측법(DXA)을 사용하여 뼈의 밀도를 측정합니다.
또한, 혈액 및 소변 검사를 통해 뼈 대사와 관련된 지표를 확인하고, 저충격 골절 병력도 중요한 진단 요소입니다.

※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※



어디가 아프신가요? (종료하

In [ ]:
# 초기화
previous_question = None # 이전 질문 저장
previous_answer = None # 이전 질문에 대한 답 저장

In [ ]:
while True:
    question = input("어디가 아프신가요? (종료하려면 '종료' 입력): ")

    # 종료 조건
    if question == "종료":
        print("프로그램을 종료합니다.")
        break

    context = "" # 이전 질문과 답변을 저장할 변수
    if previous_question and previous_answer:
        context = f"이전 질문: {previous_question}\n이전 답변: {previous_answer}\n\n" # 이전 질문과 답이 있다면 context에 넣기

    # 질문에 대해 답변을 얻음
    response = rag_chain.invoke(context + question) # 위에서 받은 context와 새로운 질문을 포함해서 질문을 넣음
    content = response.content  # content만 추출

    # 온점을 기준으로 문장을 나누고, 온점 뒤에 줄바꿈 추가
    formatted_content = "\n".join([sentence.strip() + '.' for sentence in content.split('.') if sentence.strip()])

    # 출력
    print("\n답변:\n")
    print(formatted_content)
    print("\n※ 위 진단은 정확하지 않을 수 있습니다. 정확한 진단은 의사에게 진단 받으세요. ※\n")
    print("\n==============================\n")

    # 이전 질문과 답변을 저장
    previous_question = question
    previous_answer = formatted_content


어디가 아프신가요? (종료하려면 '종료' 입력): 종료
프로그램을 종료합니다.


In [ ]:
# # PDF 파일 경로
# pdf_path = "/content/dat 프로젝트.pdf"

# # 텍스트 및 이미지 저장
# extracted_text = []

# # PDF 열기 및 텍스트 추출
# with fitz.open(pdf_path) as pdf_document:
#     for page_num in range(pdf_document.page_count):
#         page = pdf_document[page_num]

#         # 텍스트 추출
#         text = page.get_text()
#         extracted_text.append(text)

#         # 텍스트 출력
#         print(f"Page {page_num + 1} Text:\n{text}\n")

# # 모든 텍스트를 하나로 합치기
# context = "\n\n".join(extracted_text)

Page 1 Text:
 
 
골다공증 (Osteoporosis) 
1. 정의 
골다공증은 뼈의 밀도와 강도가 감소하여 뼈가 약해지고 쉽게 부러지는 질환입니다. 정
상적인 뼈 구조가 점차적으로 소실되면서 골절의 위험이 크게 증가합니다. 주로 척추, 고
관절(엉덩이뼈), 손목 등에서 골절이 잘 발생하며, 노인들에게 흔히 발생하는 만성 질환
입니다. 
2. 원인 
골다공증은 뼈를 형성하는 과정과 뼈를 흡수하는 과정의 불균형으로 발생합니다. 주요 
원인은 다음과 같습니다: 
(노화: 나이가 들수록 뼈의 형성과 흡수 사이의 균형이 깨져 뼈가 약해짐, 호르몬 변화: 
폐경 후 여성에서 에스트로겐 감소로 뼈 손실 증가, 영양 결핍: 칼슘과 비타민 D 섭취 
부족, 운동 부족: 체중 부하 운동의 부족으로 뼈 강도 저하, 유전적 요인: 가족 중 골다공
증 병력이 있는 경우, 흡연 및 음주: 뼈 형성을 방해하고 골 손실을 촉진, 질병 및 약물: 
갑상선 기능 항진증, 류마티스 관절염, 만성 신장질환 및 스테로이드 장기 복용) 
3. 증상 
골다공증은 조기에는 증상이 거의 없으며, “침묵의 질환”으로 불립니다. 그러나 뼈 손실
이 진행되면 다음과 같은 증상이 나타날 수 있습니다: 
(골절(특히 척추, 손목, 고관절 골절), 키가 감소 (척추 압박 골절로 인한 경우), 허리 및 
등 통증, 구부정한 자세(척추 후만증), 뼈가 약해져 작은 충격에도 골절 발생) 
4. 진단 방법 
골다공증의 진단은 여러 방법을 통해 이루어집니다. 가장 기본적인 검사는 **골밀도 검
사(BMD)**로, 이중 에너지 X선 흡수계측법(DXA)을 사용하여 뼈의 밀도를 측정합니다. 
DXA는 주로 척추(요추)와 고관절 부위를 검사하며, 손목 등의 골밀도도 평가할 수 있습
니다. 검사 결과는 T-점수로 나타나며, -1.0 이상은 정상, -1.0에서 -2.5 사이는 골감소증, -
2.5 이하는 골다공증으로 진단됩니다. 골밀도 검사는 골다공증 위험이 있는 사람들에게 
정기적으로 시행될 것을 권장합니다. 
골절 병